In [56]:
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score
import pandas as pd  
import numpy as np
import pickle
import codecs
import spacy
import glob
import re

all_files = glob.glob("./corpus/groupe*.txt")
df = pd.DataFrame()
for file_ in all_files: 
    list_of_lists = []
    with open(file_, 'rU') as f:
        for line in f:
            tweet = ' '.join(line.split(")")[1:])
            id = line.split(")")[0].split(",")[0][1:]
            sentiment = line.split(")")[0].split(",")[1]
            try:
                group = line.split(")")[0].split(",")[2]
            except:
                group = ''
            list_of_lists.append([id, sentiment, group, tweet])
    next_df = pd.DataFrame(list_of_lists, columns=['Id','Sentiment', 'Group', 'Tweet'])
    df = df.append(next_df)
df = df.drop_duplicates()
df = df.reset_index(drop=True)

/home/sp/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: DeprecationWarning: 'U' mode is deprecated


In [57]:
contraction_mapping = {
    "gôche": "gauche",
    "gauchos": "gauche",
    "réac": "réaction",
    "co": "companie",
    "cie": "companie",
    "and": "et",
    "but": "mais",
    "ui": "oui",
    "@macron": "Macron",
    "@EmmanuelMacron": "Emmanuel Macron",
    "GJ": "gilet jaune",
    " 1 ": " un ",
    "PR": "président",
    "RT": "retweet",
    "LREM": "la république en marche",
    "LAREM": "la république en marche",
    "#LREM": "la république en marche",
    "#LAREM": "la république en marche",
    "telma": "tellement",
    "ctr": "contre",
    "5e": "cinquieme",
    "foute": "foutre",
    "my god": "mon dieu",
    "nn": "non",
    "b8en": "bien",
    "good": "bien",
    "bad": "mauvais",
    "lui": "il"
}

In [58]:
nlp = spacy.load('fr')

def tweet_cleaner(text):
    decoded = str(text)
    apostrophe_handled = re.sub("’", "'", decoded)
    expanded = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in apostrophe_handled.split(" ")])
    parsed = nlp(expanded)
    final_tokens = []
    for t in parsed:
        if t.is_punct or t.is_space or t.like_num or t.like_url or str(t).startswith('@'):
            pass
        else:
            if t.lemma_ == '-PRON-':
                final_tokens.append(str(t))
            else:
                sc_removed = re.sub("[^a-zA-Zéèêùûàâœçî]", '', str(t.lemma_))
                if len(sc_removed) > 1:
                    final_tokens.append(sc_removed)
    joined = ' '.join(final_tokens)
    spell_corrected = re.sub(r'(.)\1+', r'\1\1', joined)
    return spell_corrected

In [59]:
df['Clean_tweet'] = [tweet_cleaner(t) for t in df.Tweet]

In [60]:
#Get stopwords from file
with open('./stop_words_fr.txt') as f:
    stopwords = f.read().splitlines()

In [61]:
X = df.Clean_tweet.values
y = df.Sentiment.values

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X)

In [62]:
def get_predictions(file):    
    #Convert the file corpus to a dataframe
    list_of_lists = []
    with open(file, 'rU') as f:
        for line in f:
            tweet = ' '.join(line.split(")")[1:])
            id = line.split(")")[0].split(",")[0][1:]
            sentiment = line.split(")")[0].split(",")[1]
            try:
                group = line.split(")")[0].split(",")[2]
            except:
                group = ''
            list_of_lists.append([id, sentiment, group, tweet])
    df = pd.DataFrame(list_of_lists, columns=['Id','Sentiment', ' ', 'Tweet'])
    print(df.info())
    
    #Clean and transform the data
    df['Clean_tweet'] = [tweet_cleaner(t) for t in df.Tweet]
    
    X = df.Clean_tweet.values
    y = df.Sentiment.values
    
    tv = TfidfVectorizer()
    X = tv.fit_transform(X)
    
    #Use the saved classifier and evaluate the predictions
    vec = open("rf_classifier.pickle", 'rb')
    clf = pickle.load(vec)
    vec.close()
    
    #Use a container for the dataframe to make the prediction
    container_df = pd.DataFrame(0.0, index=np.arange(len(X.todense())), columns=vectorizer.get_feature_names())
    df_topredict = pd.DataFrame(X.todense(), columns=tv.get_feature_names())
    for column in df_topredict:
        if column in vectorizer.get_feature_names():
            container_df[column] = df_topredict[column]

    preds = clf.predict(container_df)
    df['Predicted_sentiment'] = preds
    print(df['Predicted_sentiment'])
    
    text_file = open("Annoted_Tweets.txt", "w")
    for index, row in df.iterrows():
        line = '(' + row['Id'] + ', ' + row['Predicted_sentiment'] + ') ' + row['Tweet']
        text_file.write(line)
    text_file.close()

In [63]:
get_predictions('TweetsAboutMacron.txt')

/home/sp/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: 'U' mode is deprecated
  after removing the cwd from sys.path.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
Id           100 non-null object
Sentiment    100 non-null object
             100 non-null object
Tweet        100 non-null object
dtypes: object(4)
memory usage: 3.2+ KB
None
0     neg
1     neg
2     neg
3     neg
4     neu
5     neg
6     neg
7     neg
8     neg
9     neg
10    neg
11    neg
12    neg
13    neg
14    neg
15    neg
16    neg
17    neg
18    neg
19    neg
20    neg
21    neu
22    neg
23    neg
24    neg
25    neg
26    neg
27    neg
28    neg
29    neg
     ... 
70    neg
71    neg
72    neu
73    neg
74    neg
75    neg
76    neg
77    neg
78    neg
79    neg
80    irr
81    neg
82    neg
83    neg
84    neu
85    neu
86    neg
87    neg
88    neg
89    neg
90    neg
91    neg
92    neu
93    neg
94    neg
95    irr
96    neg
97    irr
98    neg
99    neg
Name: Predicted_sentiment, Length: 100, dtype: object


In [69]:
list_of_lists = []
with open('TweetsAboutMacron.txt', 'rU') as f:
    for line in f:
        tweet = ' '.join(line.split(")")[1:])
        id = line.split(")")[0].split(",")[0][1:]
        sentiment = line.split(")")[0].split(",")[1]
        try:
            group = line.split(")")[0].split(",")[2]
        except:
            group = ''
        list_of_lists.append([id, sentiment, group, tweet])
df = pd.DataFrame(list_of_lists, columns=['Id','Sentiment', ' ', 'Tweet'])
print(df.info())

#Clean and transform the data
df['Clean_tweet'] = [tweet_cleaner(t) for t in df.Tweet]

X = df.Clean_tweet.values
y = df.Sentiment.values

tv = TfidfVectorizer()
X = tv.fit_transform(X)

#Use the saved classifier and evaluate the predictions
vec = open("rf_classifier.pickle", 'rb')
clf = pickle.load(vec)
vec.close()

#Use a container for the dataframe to make the prediction
container_df = pd.DataFrame(0.0, index=np.arange(len(X.todense())), columns=vectorizer.get_feature_names())
df_topredict = pd.DataFrame(X.todense(), columns=tv.get_feature_names())

/home/sp/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 'U' mode is deprecated
  


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
Id           100 non-null object
Sentiment    100 non-null object
             100 non-null object
Tweet        100 non-null object
dtypes: object(4)
memory usage: 3.2+ KB
None


In [73]:
for column in df_topredict:
    if column in vectorizer.get_feature_names():
        container_df[column] = df_topredict[column]

In [75]:
len(container_df.columns)

10332

In [76]:
clf.predict(container_df)

array(['neg', 'neg', 'neg', 'neg', 'neu', 'neg', 'neg', 'neg', 'neg',
       'neg', 'neg', 'neg', 'neg', 'neg', 'neg', 'neg', 'neg', 'neg',
       'neg', 'neg', 'neg', 'neu', 'neg', 'neg', 'neg', 'neg', 'neg',
       'neg', 'neg', 'neg', 'neg', 'pos', 'neg', 'neg', 'neg', 'neg',
       'neg', 'neg', 'neg', 'neg', 'neg', 'neg', 'neg', 'neg', 'neg',
       'neg', 'neg', 'neu', 'neg', 'neg', 'neg', 'neg', 'neg', 'neg',
       'neg', 'neg', 'neu', 'neg', 'neu', 'neg', 'pos', 'neg', 'neg',
       'neg', 'pos', 'neg', 'neu', 'neg', 'neg', 'neg', 'neg', 'neg',
       'neu', 'neg', 'neg', 'neg', 'neg', 'neg', 'neg', 'neg', 'irr',
       'neg', 'neg', 'neg', 'neu', 'neu', 'neg', 'neg', 'neg', 'neg',
       'neg', 'neg', 'neu', 'neg', 'neg', 'irr', 'neg', 'irr', 'neg',
       'neg'], dtype=object)